In [ ]:
# ======================================================
# ETL PIPELINE: Superstore → Google Sheets (modelo Star)
# Visualización: Looker Studio
# ======================================================

# 1. Importamos librerías necesarias

In [ ]:
# --- 1. Librerías necesarias ---
import pandas as pd
import numpy as np
from datetime import datetime
from google.colab import auth
import gspread
from google.auth import default
from googleapiclient.discovery import build

# 2. Autenticación con Google

Usamos la librería google.auth para tener acceso a crear Documento de Google Sheets. Cada tab/hoja de cálculo, equivale a una tabla de una Base de Datos

In [ ]:
# --- 2. Autenticación con Google ---
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
print("✅ Autenticado con Google correctamente.")

# 3. Importar / Cargar Dataset en formato csv

Vamos a user el dataset Superstore
<BR>
Ver en [Kaggle](https://www.kaggle.com/datasets/vivek468/superstore-dataset-final)

Como venimos haciendo hasta ahora, descargamos el Dataset "Superstore" y lo guardamos en nuestro Google Drive. Luego lo cargamos en memoria para el procesamiento.

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("vivek468/superstore-dataset-final")

print("Path to dataset files:", path)

In [ ]:
!ls /root/.cache/kagglehub/datasets/vivek468/superstore-dataset-final/versions/1

In [ ]:
# Montar la unidad
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# En caso que necesiten hace un unmount de la unidad
# drive.flush_and_unmount

In [ ]:
# Ahora copiar el dataset descargado a nuestro Google Drive
import os, shutil
# Carpeta destino
dest_dir = "/content/drive/MyDrive/datasets/"

# Copiar todos los archivos del dataset al destino
for file in os.listdir(path):
    shutil.copy(os.path.join(path, file), dest_dir)

# print("Archivos copiados a:", dest_dir)
# print("Contenido:", os.listdir(dest_dir))

os.listdir(dest_dir)

In [ ]:
# Usar el método read_csv de Pandas (pd) que recibe como argumento el archivo csv y retorna un dataframe
os.chdir('/content/drive/MyDrive/datasets')
df = pd.read_csv("Sample - Superstore.csv", encoding="latin1")

In [ ]:
df.sample()

# 4. Limpieza de datos

Asi como trabajamos en nuestra Pre-entrega, realizamos una limpieza de duplicados y nulos para alimentar con datos de calidad la próxima etapa.

In [ ]:
# Normalizar nombres de columnas
df.columns = df.columns.str.strip().str.replace(" ", "_").str.lower()

In [ ]:
df.columns

In [ ]:
# --- 4. Limpieza básica ---
df = df.dropna(subset=["order_date", "sales", "profit"])
df["order_date"] = pd.to_datetime(df["order_date"])
df["ship_date"] = pd.to_datetime(df["ship_date"])

In [ ]:
# Crear campos derivados
df["profit_margin"] = df["profit"] / df["sales"]
df["order_year"] = df["order_date"].dt.year
df["order_month"] = df["order_date"].dt.month
df["order_quarter"] = df["order_date"].dt.quarter
df["order_day"] = df["order_date"].dt.day
df["order_week"] = df["order_date"].dt.isocalendar().week.astype(int)

print("🧹 Limpieza y enriquecimiento de datos completado.")

# 5. Construcción de las dimensiones (modelo estrella)

Este es un concepto nuevo. Para poder representar los datos en Looker Studio (u otra herramienta de visualización), necesitamos que nuestros datos se encuentren organizados en una Base de Datos Operacional (OLAP). Esta base consta de una tabla de "facts" o "hechos" con métricas que luego se puede agregar (sumar, promediar, obtener máximos, etc) y tantan tablas de dimensiones como necesitemos. Estas tablas de dimensiones nos permiten analizar los datos desde múltiples perspectivas (como si fuera un cubo, donde cada lado se corresponde con una dimensión). Las dimensiones más comunes son: la tempora y la geográfica, en nuestro caso sumamos además la de categorías y de clientes.

A continuación cremos primeros las tablas correspondientes a cada dimensión. En nuestro caso usaremos tablas "desnomarmalizadas", y cada una tendra una clave primaria, que será referenciada desde la tabla de hechos, o fact table, que veremos debajo.

In [ ]:
# Dimensión Fecha
dim_date = df[["order_date", "order_year", "order_quarter", "order_month", "order_week", "order_day"]] \
    .drop_duplicates().reset_index(drop=True)
dim_date.insert(0, "date_id", range(1, len(dim_date)+1))

In [ ]:
# Dimensión Producto
dim_product = df[["category", "sub-category", "product_name"]] \
    .drop_duplicates().reset_index(drop=True)
dim_product.insert(0, "product_id", range(1, len(dim_product)+1))

In [ ]:
# Dimensión Geografía
dim_geo = df[["country", "region", "state", "city"]] \
    .drop_duplicates().reset_index(drop=True)
dim_geo.insert(0, "geo_id", range(1, len(dim_geo)+1))

In [ ]:
# Dimensión Cliente
dim_customer = df[["customer_id", "customer_name", "segment"]] \
    .drop_duplicates().reset_index(drop=True)
dim_customer.insert(0, "customer_sk", range(1, len(dim_customer)+1))

print("📚 Dimensiones creadas:")
print(f"  dim_date={len(dim_date)}, dim_product={len(dim_product)}, dim_geo={len(dim_geo)}, dim_customer={len(dim_customer)}")

# 6. Construcción de la Tabla de Hechos (o bien conocida como Fact Table)

Esta es la tabla que mencionamos arteriormente, que solo contiene las métricas y las Claves Primarias a las tablas de dimensiones.

In [ ]:
# Mapear claves surrogate
date_map = dict(zip(dim_date["order_date"], dim_date["date_id"]))
prod_map = dict(zip(dim_product["product_name"], dim_product["product_id"]))
geo_map = dict(zip(dim_geo["city"], dim_geo["geo_id"]))
cust_map = dict(zip(dim_customer["customer_id"], dim_customer["customer_sk"]))

fact_sales = pd.DataFrame({
    "order_id": df["order_id"],
    "date_id": df["order_date"].map(date_map),
    "product_id": df["product_name"].map(prod_map),
    "geo_id": df["city"].map(geo_map),
    "customer_id": df["customer_id"].map(cust_map),
    "sales": df["sales"],
    "profit": df["profit"],
    "quantity": df["quantity"],
    "discount": df["discount"],
    "profit_margin": df["profit_margin"]
})

print(f"🧾 Tabla de hechos creada: {len(fact_sales)} registros.")

# 7. Poblar las tablas de un Documento de Google Sheets

Procedemos ahora a "poblar" nuestras tablas. En este caso usamos hojas de cálculo de Google Sheets, que luego podemos reemplazar por una Base de Datos Relacional, como ser PostgreSQL.

In [ ]:
# Buscamos el archivo "DW_Superstore_OLAP_Star"
# Si existe lo reutilizamos, sino lo creamos.
# La idea es que el ETL sea reproducible

# ----------------------------------------------------------
# Buscar o crear el archivo en Google Drive
service = build('drive', 'v3', credentials=creds)

file_name = "DW_Superstore_OLAP"
results = service.files().list(
    q=f"name='{file_name}' and mimeType='application/vnd.google-apps.spreadsheet'",
    spaces='drive'
).execute()

if results['files']:
    spreadsheet_id = results['files'][0]['id']
    spreadsheet = gc.open_by_key(spreadsheet_id)
    print("📂 Archivo existente encontrado:", spreadsheet.url)
else:
    spreadsheet = gc.create(file_name)
    print("🆕 Archivo nuevo creado:", spreadsheet.url)

# ----------------------------------------------------------
# Mantener "Hoja 1" intacta y limpiar las demás hojas antes de reescribirlas
worksheets = spreadsheet.worksheets()
existing_titles = [ws.title for ws in worksheets]

print("📄 Hojas existentes:", existing_titles)

# ----------------------------------------------------------
# Definir las tablas OLAP a subir
tables = {
    "dim_date": dim_date,
    "dim_product": dim_product,
    "dim_geo": dim_geo,
    "dim_customer": dim_customer,
    "fact_sales": fact_sales
}

# ----------------------------------------------------------
# Subir las tablas sin eliminar hojas (solo limpiar contenido)
for name, df_table in tables.items():
    # Convertir columnas datetime a string
    for col in df_table.columns:
        if np.issubdtype(df_table[col].dtype, np.datetime64):
            df_table[col] = df_table[col].dt.strftime('%Y-%m-%d')

    # Si la hoja ya existe, limpiar su contenido; si no, crearla
    existing_titles = [ws.title for ws in spreadsheet.worksheets()]
    if name in existing_titles:
        ws = spreadsheet.worksheet(name)
        ws.clear()
        print(f"🧹 Hoja existente '{name}' limpiada.")
    else:
        ws = spreadsheet.add_worksheet(
            title=name,
            rows=str(len(df_table) + 1),
            cols=str(len(df_table.columns) + 1)
        )
        print(f"🆕 Hoja '{name}' creada.")

    # Subir los datos (encabezados + valores)
    ws.update([df_table.columns.values.tolist()] + df_table.astype(str).values.tolist())
    print(f"📈 Hoja '{name}' actualizada correctamente ({len(df_table)} filas).")

print("✅ Data Warehouse OLAP actualizado correctamente.")
print("📊 URL:", spreadsheet.url)



Comprobar que las hojas del Google Sheet se completaron y continuar el armado del Dashboard desde Looker Studio.
<BR>
Como primer paso, se importan las tablas. Luego, a partir de los KPIs (primary key indicators) se arman los join y luego se agregan los gráficos y se configuran en el panel de propiedades.

# Validaciones

Aquí podemos hacer join/merge y agregaciones para validar los datos que arroja Looker Studio.

In [ ]:
fact_sales["customer_id"]=fact_sales["customer_id"].astype(str)
dim_customer["customer_sk"]=dim_customer["customer_sk"].astype(str)
facts_customer = pd.merge(fact_sales, dim_customer, left_on="customer_id", right_on="customer_sk", how="left")

In [ ]:
facts_customer.head()

In [ ]:
facts_customer.columns

In [ ]:
facts_customer.groupby("customer_name").agg(
    {"sales":"sum",
     "profit": "sum"}
)